In [2]:
#from google.colab import files
#uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [1]:
import numpy as np
import pandas as pd
import csv
import io

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from collections import Counter
from pandas import DataFrame
import nltk

%matplotlib inline

from sklearn.model_selection import GridSearchCV

In [4]:
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#!pip install category_encoders

     |████████████████████████████████| 81kB 2.2MB/s 


In [7]:
import category_encoders as ce

In [8]:
sample = pd.read_csv(io.BytesIO(uploaded['sample_submission.csv']))
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [9]:
train['exclamacion'] = train['text'].str.count("!")
train['at'] = train['text'].str.count("@")

In [10]:
english_stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')

aQuitar = ['http','https','&amp']
english_stopwords.extend(aQuitar)

In [11]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [12]:
texto = train["text"]

textoVeridico = train[(train['target']==1)]
textoVeridico = textoVeridico["text"]

textoFalso = train[(train['target']==0)]
textoFalso = textoFalso["text"]

In [13]:
palabrasVeridicas=[]
for row in textoVeridico:
    row =row.lower()
    row=filter(None, re.split("[, \-!?:._¡¿'`<>;()+]", row))
    for palabra in row:
        if len(palabra)<4: #saco conectores
            continue
        if palabra not in english_stopwords:
            palabrasVeridicas.append(palabra)

In [14]:
palabrasFalsas=[]
for row in textoFalso:
    row =row.lower()
    row=filter(None, re.split("[, \-!?:._¡¿'`<>;()+]", row))
    for palabra in row:
        if len(palabra)<4: #saco conectores
            continue
        if palabra not in english_stopwords:
            palabrasFalsas.append(palabra)

In [15]:
stems = stem_tokens(palabrasVeridicas, stemmer)
contador = Counter(stems)
palabrasComunesVeridicas = contador.most_common(100)
dfPalabrasComunesV = DataFrame(palabrasComunesVeridicas,columns=['Palabra','Cantidad'])

In [16]:
stems = stem_tokens(palabrasFalsas, stemmer)
contador = Counter(stems)
palabrasComunesFalsas = contador.most_common(100)
dfPalabrasComunesF = DataFrame(palabrasComunesFalsas,columns=['Palabra','Cantidad'])

In [17]:
popularesEnComun = dfPalabrasComunesV.merge(dfPalabrasComunesF,left_on='Palabra',right_on='Palabra',how='inner')

In [18]:
popularesExclusivamenteV = dfPalabrasComunesV[(~dfPalabrasComunesV.Palabra.isin(popularesEnComun.Palabra))]
popularesExclusivamenteF = dfPalabrasComunesF[(~dfPalabrasComunesF.Palabra.isin(popularesEnComun.Palabra))]

In [ ]:
TopPopularesExclusivamenteV = popularesExclusivamenteV.nlargest(10, 'Cantidad')
TopPopularesExclusivamenteV.sort_values('Cantidad',inplace=True)

In [ ]:
TopPopularesExclusivamenteF = popularesExclusivamenteF.nlargest(10, 'Cantidad')
TopPopularesExclusivamenteF.sort_values('Cantidad',inplace=True)

In [25]:
for columna in popularesExclusivamenteV['Palabra']:
    train[columna+'_text']=train.text.str.contains(columna,case=False)

In [26]:
for columna in popularesExclusivamenteF['Palabra']:
    train[columna+'_text']=train.text.str.contains(columna,case=False)

In [27]:
train = train.drop('id', axis=1)
train = train.drop('text', axis=1)

In [28]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

In [29]:
train = oneHotEncoding(train,'keyword')

In [30]:
target_enc = ce.TargetEncoder(train['location'])

# Fit the encoder using the categorical features and target
target_enc.fit(train['location'], train['target'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc.transform(train['location']).add_suffix('_target'))

train = train.drop('location', axis=1)

In [31]:
train_label = []
train_data = []

train_label = train["target"]
train_data = train.iloc[:, train.columns != 'target']

del train

In [32]:
IDS=test[['id']].copy()

In [33]:
test['exclamacion'] = test['text'].str.count("!")
test['at'] = test['text'].str.count("@")

In [34]:
for columna in popularesExclusivamenteV['Palabra']:
    test[columna+'_text']=test.text.str.contains(columna,case=False)

In [35]:
for columna in popularesExclusivamenteF['Palabra']:
    test[columna+'_text']=test.text.str.contains(columna,case=False)

In [36]:
test = test.drop('id', axis=1)
test = test.drop('text', axis=1)

In [37]:
test = oneHotEncoding(test,'keyword')

In [38]:
test = test.join(target_enc.transform(test['location']).add_suffix('_target'))
test = test.drop('location', axis=1)
del target_enc

In [39]:
#-----------------------------------------------------------------
# Modelo Random Forest Classifier
#-----------------------------------------------------------------
from sklearn.ensemble import RandomForestClassifier

In [40]:
modeloRFR=RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=150)

In [41]:
modeloRFR.fit(train_data, train_label)

result = modeloRFR.predict(test)

In [42]:
sample.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [43]:
entregable_RFR=IDS[['id']].copy()
entregable_RFR['target']=result

In [44]:
entregable_RFR.head()

,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1


In [45]:
entregable_RFR['target'].value_counts()

0    1932
1    1331
Name: target, dtype: int64

In [46]:
entregable_RFR.to_csv('RFR_masPalabas.csv', index=False)